<a href="https://colab.research.google.com/github/bhagesh-codebeast/VideoTranscribeTranslate/blob/main/VideoTranscribeandTranslate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Video Transcribe and Translate

In [ ]:
!python --version

In [ ]:
!pipx ensurepath
!apt install pipx
!pip install pydub
!pip install pytube
!pip install moviepy
!pip install -U deep-translator
!pipx install insanely-fast-whisper --force
!pip install -Uqq WhisperSpeech

In [ ]:
from deep_translator import GoogleTranslator
GoogleTranslator().get_supported_languages(as_dict=True)

In [18]:
import os
import re
import json
import glob
import torch
import subprocess
from pytube import YouTube
from datetime import datetime
from pydub import AudioSegment
import torch.nn.functional as F
from moviepy.editor import VideoFileClip
from whisperspeech.pipeline import Pipeline
from deep_translator import GoogleTranslator


class videoTranscribeandTranslate:
  def __init__(self, input_video_path='',  ifw_path='', output_audio_path=os.path.join(os.getcwd(),'downloaded_audio.mp3'), ifw_transcript=os.path.join(os.getcwd(),'downloaded_timestamp.json'),source='auto',language='en',speaker=os.path.join(os.getcwd(),'downloaded_audio.mp3'),translated_audio=os.path.join(os.getcwd(),'translated_audio.mp3'), custom_text=''):
    date = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
    self.input_video_path = input_video_path
    self.output_audio_path = output_audio_path.split('.')[0]+'_'+date+'.'+output_audio_path.split('.')[1]
    self.ifw_transcript = ifw_transcript.split('.')[0]+'_'+date+'.'+ifw_transcript.split('.')[1]
    self.ifw_path = ifw_path
    self.source = source
    self.language = language
    self.speaker = speaker
    self.translated_audio = translated_audio.split('.')[0]+'_'+date+'.'+translated_audio.split('.')[1]
    self.custom_text = custom_text
  def getAudio(self):
    if self.input_video_path.startswith('https'):
      YouTube(self.input_video_path).streams.filter(only_audio=True, file_extension='mp4').first().download(filename=self.output_audio_path)
    elif os.path.exists(self.input_video_path):
      video_clip = VideoFileClip(self.input_video_path)
      audio_clip = video_clip.audio
      audio_clip.write_audiofile(self.output_audio_path, codec='mp3')
    return self.output_audio_path
  def transcribeAudio(self):
    if os.path.exists(self.getAudio()):
      command = [self.ifw_path, "--transcript-path", self.ifw_transcript, "--file-name", self.output_audio_path]
      result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
      if result.returncode == 0:
        return self.ifw_transcript
  def getText(self):
    self.transcribeAudio()
    if os.path.exists(self.ifw_transcript):
      text = json.loads(open(self.ifw_transcript, "r").read()).get("text", "")
      return text
  def translateText(self):
    string = self.getText()
    translated = {}
    if len(string) > 3000:
      result_list = [string[i:i+2000] if i+2000 >= len(string) else string[i:i+2000].rsplit(' ', 1)[0] for i in range(0, len(string), 2000)]
    else:
      result_list = [string]
    for text in result_list:
      translated[text] = {GoogleTranslator(source=self.source, target=self.language).translate(text=str(text))}
    return translated
  def changeAudio(self):
    if self.language == 'en':
      result_list = []
      if self.custom_text:
        string = self.custom_text
        if len(string) > 3000:
          result_list = [string[i:i+2000] if i+2000 >= len(string) else string[i:i+2000].rsplit(' ', 1)[0] for i in range(0, len(string), 2000)]
        else:
          result_list = [string]
      else:
        result_list = self.translateText().values()
        if not len(result_list) > 0:
          result_list = [self.getText()]
      # Q4 tiny model
      # pipe = Pipeline(s2a_ref='collabora/whisperspeech:s2a-q4-tiny-en+pl.model')
      # Q4 small model
      pipe = Pipeline()
      for i, item in enumerate(result_list, 1):
        filename = os.path.join(os.path.dirname(self.translated_audio),f"temp_{i}.mp3")
        pipe.generate_to_file(filename, f"""{item}""", lang=self.language, speaker=self.speaker)
      mp3_files = glob.glob(os.path.join(os.path.dirname(self.translated_audio),"temp_*.mp3"))
      mp3_files.sort(key=lambda x: int(x.split("_")[1].split(".")[0]))
      merged_audio = AudioSegment.silent()
      for mp3_file in mp3_files:
        audio_segment = AudioSegment.from_file(mp3_file, format="mp3")
        merged_audio += audio_segment
      merged_audio.export(self.translated_audio, format="mp3")
      for mp3_file in mp3_files:
        os.remove(mp3_file)
      os.remove(self.input_video_path)
      return self.translated_audio


In [21]:
from IPython.display import Audio, display

ifw_path = '/root/.local/pipx/venvs/insanely-fast-whisper/bin/insanely-fast-whisper'

## Get Audio (Speaker)

In [ ]:
%%time
voiceover = 'https://www.youtube.com/watch?v=gA-FA2a6ESI'
instance2 = videoTranscribeandTranslate(voiceover, ifw_path, output_audio_path=os.path.join(os.getcwd(),'voiceover_audio.mp3'))
speaker = instance2.getAudio()

display(Audio(speaker, autoplay=True))

## Convert Text to Audio

In [ ]:
%%time
in_text = 'the temple town of tirupati in chitturu district of Andra Pradesh, its existance to the temple of lord Shri Venkateshwara on the the Tirumala hill nextling among the 7 hills in the Eastern Ghats.'
instance3 = videoTranscribeandTranslate(ifw_path, speaker=speaker, custom_text=in_text)

display(Audio(instance3.changeAudio(), autoplay=True))

## Change Audio

In [ ]:
%%time
input_video = 'https://www.youtube.com/watch?v=p8QOnty6rSU'
instance = videoTranscribeandTranslate(input_video, ifw_path, speaker=speaker)

display(Audio(instance.changeAudio(), autoplay=True))

# References
### 1. [insanely-fast-whisper](https://github.com/Vaibhavs10/insanely-fast-whisper)
### 2. [Youtube Summariser](https://github.com/jxcinta/youtube_summariser/blob/main/youtube_summariser.py)
### 3. [WhisperSpeech](https://github.com/collabora/WhisperSpeech/tree/main)